In [12]:
import os
import random
import shutil
from PIL import Image
from glob import glob
from shutil import copyfile
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

In [2]:
normal = os.listdir('/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/Normal/images')
print('Normal Cases', len(normal))

covid = os.listdir('/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/COVID/images')
print('covid Cases', len(covid))

Lung_Op = os.listdir('/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/Lung_Opacity/images')
print('Lung_Op Cases', len(Lung_Op))

Pneumonia = os.listdir('/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/Viral Pneumonia/images')
print('Pneumonia Cases', len(Pneumonia))

Normal Cases 10192
covid Cases 3616
Lung_Op Cases 6012
Pneumonia Cases 1345


In [3]:
def sample_data(source, root_dir, cls):
    test_ratio = .25
    val_ratio = 0.15
    all_ = os.listdir(source)
    all_files = random.sample(all_, 800)
    np.random.shuffle(all_files)
    train_FileNames, test_FileNames, val_FileNames = np.split(np.array(all_files),
                                                                    [int(len(all_files) * (1 - (val_ratio + test_ratio))),
                                                                    int(len(all_files) * (1 - val_ratio)),
                                                                    ])

    train_FileNames = [source + '/' + name for name in train_FileNames.tolist()]
    val_FileNames = [source + '/' + name for name in val_FileNames.tolist()]
    test_FileNames = [source + '/' + name for name in test_FileNames.tolist()]
    print('Total images: '+ str(len(all_files)))
    print('Training: '+ str(len(train_FileNames)))
    print('Validation: '+  str(len(val_FileNames)))
    print('Testing: '+ str(len(test_FileNames)))

    os.makedirs(root_dir + '/train/' + cls)
    os.makedirs(root_dir + '/val/' + cls)
    os.makedirs(root_dir + '/test/' + cls)

    for name in train_FileNames:
        shutil.copy(name, root_dir + '/train/' + cls)

    for name in val_FileNames:
        shutil.copy(name, root_dir + '/val/' + cls)

    for name in test_FileNames:
        shutil.copy(name, root_dir + '/test/' + cls)

In [28]:
# Original Image paths - Names
train_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/research_data/train/COVID/'
val_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/research_data/val/COVID/'
test_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/research_data/test/COVID/'


train_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/research_data/train/normal/'
val_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/research_data/val/normal/'
test_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/research_data/test/normal/'

original_file_paths = [train_covid, val_covid, test_covid, train_normal, val_normal, test_normal]

In [29]:
# masks Images paths - Source
train_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/COVID/masks/'
val_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/COVID/masks/'
test_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/COVID/masks/'


train_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/Normal/masks/'
val_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/Normal/masks/'
test_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/Normal/masks/'

masks_file_paths_src = [train_covid, val_covid, test_covid, train_normal, val_normal, test_normal]

In [30]:
# masks Images paths - Destination
train_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/masked_images/train/COVID/'
val_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/masked_images/val/COVID/'
test_covid = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/masked_images/test/COVID/'


train_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/masked_images/train/normal/'
val_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/masked_images/val/normal/'
test_normal = '/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset/masked_images/test/normal/'

masks_file_paths_dest = [train_covid, val_covid, test_covid, train_normal, val_normal, test_normal]

In [47]:
for i in range(len(masks_file_paths_dest)):
    files = os.listdir(original_file_paths[i])
    FileNames = [masks_file_paths_src[i] + name for name in files]
    for name in files:
        shutil.copy(masks_file_paths_src[i] + name, masks_file_paths_dest[i] + name, follow_symlinks=True)

In [31]:
for i in range(len(masks_file_paths_dest)):
    files = os.listdir(original_file_paths[i])
    for name in files:
        image1 = Image.open(original_file_paths[i] + name).convert('RGB')
        image2 = Image.open(masks_file_paths_src[i] + name).convert('RGB')

        img1_array = np.array(image1.resize((224, 224)))
        img2_array = np.array(image2.resize((224, 224)))

        result_array = np.multiply(img1_array, img2_array)
        result_image = Image.fromarray(result_array)
        
        result_image.save(masks_file_paths_dest[i] + name)

In [36]:
# len(os.listdir('COVID-19_Radiography_Dataset/Viral Pneumonia/images'))

1345

In [21]:
source = 'COVID-19_Radiography_Dataset/COVID/images'
root_dir = 'COVID-19_Radiography_Dataset/research_data'
cls = "COVID"
sample_data(source, root_dir, cls)

Total images: 800
Training: 480
Validation: 120
Testing: 200


In [22]:
source = 'COVID-19_Radiography_Dataset/Normal/images'
root_dir = 'COVID-19_Radiography_Dataset/research_data'
cls = "normal"
sample_data(source, root_dir, cls)

Total images: 800
Training: 480
Validation: 120
Testing: 200


In [9]:
def split_data(source, train_path, test_path, split_size, split_type):
    if split_type == "normal": 
        all_files=os.listdir(source)
        down_Sample_files=random.sample(all_files, int(len(all_files) * 0.36))
        train_files=random.sample(down_Sample_files, int(len(down_Sample_files) * split_size))
    else:
        all_files=os.listdir(source)
        train_files=random.sample(all_files, int(len(all_files) * split_size))
    for f in train_files: copyfile(source + f, train_path + f)
    for f in os.listdir(source):
        if split_type == "normal": 
            if f not in train_files and f in down_Sample_files:
                copyfile(source + f, test_path + f)
        else:
            if f not in train_files:
                copyfile(source + f, test_path + f)

In [74]:
# NORMAL_DATA_PATH = BASE_PATH + "/Normal/images/"
# COVID_DATA_PATH = BASE_PATH + "/COVID/images/"

# Test to check if paths are correct and working
# covid_files = len(os.listdir(COVID_DATA_PATH))
# normal_files = len(os.listdir(NORMAL_DATA_PATH))

# print(f"COVID-19 Xray Scans: {covid_files}")
# print(f"Normal Xray Scans: {normal_files}")

# TRAIN_NORMAL_PATH = BASE_PATH + "/data/training/normal/"
# TRAIN_COVID_PATH = BASE_PATH + "/data/training/covid/"
# TEST_NORMAL_PATH = BASE_PATH + "/data/testing/normal/"
# TEST_COVID_PATH = BASE_PATH + "/data/testing/covid/" 

# train_normal_files = len(os.listdir(TRAIN_NORMAL_PATH))
# train_covid_files = len(os.listdir(TRAIN_COVID_PATH))

# test_normal_files = len(os.listdir(TEST_NORMAL_PATH))
# test_covid_files = len(os.listdir(TEST_COVID_PATH))

# print(f"Training Set - Normal: {train_normal_files}")
# print(f"Training Set - Covid: {train_covid_files}")

# print(f"Testting Set - Normal: {test_normal_files}")
# print(f"Testting Set - Covid: {test_covid_files}")

In [ ]:
# SPLIT_SIZE = 0.8
# split_data(NORMAL_DATA_PATH, TRAIN_NORMAL_PATH, TEST_NORMAL_PATH, SPLIT_SIZE, 'normal')
# split_data(COVID_DATA_PATH, TRAIN_COVID_PATH, TEST_COVID_PATH, SPLIT_SIZE, "covid")

In [4]:
pred = np.load('pred_2024-09-12 06:51:12.264180.npz')['arr_0']

In [5]:
def compute_aucs(predictions, labels):
    num_classes = predictions.shape[1]
    aucs = np.zeros(num_classes)
    for i in range(num_classes):
        aucs[i] = roc_auc_score(labels[:, i], predictions[:, i])
    return aucs

In [41]:
INPUT_SHAPE = 224
VALIDATION_BATCH_SIZE = 40
BASE_PATH = "/home/ssehg1@cfreg.local/covid_detection/COVID-19_Radiography_Dataset"
TEST_DIR = BASE_PATH + "/research_data/test"
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(TEST_DIR, batch_size = VALIDATION_BATCH_SIZE,
                                                target_size = (INPUT_SHAPE, INPUT_SHAPE), class_mode = "categorical", shuffle=False)

Found 400 images belonging to 2 classes.


In [42]:
labels = test_generator.labels

In [ ]:
y_pred = np.argmax(pred, axis=1)
y_pred

In [ ]:
y_pred = np.where(pred>0.5, 1, 0)
y_pred

In [ ]:
confusion_matrix(labels, y_pred)

In [37]:
print(classification_report(labels, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.20      0.33       200
           1       0.55      0.97      0.70       200

    accuracy                           0.59       400
   macro avg       0.71      0.59      0.52       400
weighted avg       0.71      0.59      0.52       400



In [52]:
compute_aucs(np.argmax(pred, axis=1).reshape(400, 1), labels.reshape(400, 1))

array([0.7625])

In [10]:
from tensorflow.keras.applications import DenseNet121, ResNet152, EfficientNetB0
from tensorflow.keras.applications.vgg19 import VGG19

In [11]:
# base_model = EfficientNetB0(
#             include_top=False,
#             weights="imagenet",
#             input_shape=(224, 224, 3),)

base_model = VGG19(
            include_top=False,
            weights="imagenet",
            input_shape=(224,224,3),
            # pooling="max",
        )

In [13]:
print(base_model.summary())

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [21]:
len(base_model.layers[0:25])

22